# Day 2: PID Tuning vs MPC for Lane Keeping


## 3. Introduction to Model Predictive Control (MPC)

- MPC optimizes control inputs by predicting future states over a **prediction horizon**.
- Uses a cost function, e.g.:

$$
J = \sum_{k=0}^N (y_{ref}(k) - y(k))^2 + \lambda \, u(k)^2
$$

Where:
- $y_{ref}(k)$ = desired trajectory (lane center).
- $y(k)$ = predicted position.
- $u(k)$ = steering input.
- $N$ = prediction horizon.

MPC handles **constraints** (e.g., steering limits, lane boundaries).



## 1. PID Control Refresher

The PID controller computes control output:

$$
u(t) = K_p \, e(t) + K_i \int e(t)\, dt + K_d \frac{de(t)}{dt}
$$

Where:
- $e(t)$ = lane offset error (m).
- $u(t)$ = steering command.

### Tuning Methods
- **Ziegler–Nichols**: increase $K_p$ until oscillation, then compute $K_i, K_d$.
- **Manual**: start with $K_p$, then add $K_d$, then $K_i$.


## 2. Python Simulation: PID Lane Keeping

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Simulation params
dt = 0.1
T = 20
time = np.arange(0, T, dt)
lane_offset = []
steering = []

# PID gains
Kp, Ki, Kd = 0.3, 0.02, 0.1

# Initial conditions
offset = 1.0   # start 1 meter off-center
integral = 0
prev_error = offset

for t in time:
    # PID control
    error = offset
    integral += error * dt
    derivative = (error - prev_error) / dt
    steer = Kp*error + Ki*integral + Kd*derivative

    # Vehicle response (simplified)
    offset -= steer * dt

    # Store results
    lane_offset.append(offset)
    steering.append(steer)
    prev_error = error

# Plot results
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(time, lane_offset, label="Lane Offset (m)")
plt.xlabel("Time (s)")
plt.ylabel("Offset (m)")
plt.title("PID Lane Offset Correction")
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)
plt.plot(time, steering, label="Steering Command")
plt.xlabel("Time (s)")
plt.ylabel("Steer (arb. units)")
plt.title("PID Steering Output")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()



## 3. Introduction to Model Predictive Control (MPC)

- MPC optimizes control inputs by predicting future states over a **prediction horizon**.
- Uses a cost function, e.g.:

$$
J = \sum_{k=0}^N (y_{ref}(k) - y(k))^2 + \lambda \, u(k)^2
$$

Where:
- $y_{ref}(k)$ = desired trajectory (lane center).
- $y(k)$ = predicted position.
- $u(k)$ = steering input.
- $N$ = prediction horizon.

MPC handles **constraints** (e.g., steering limits, lane boundaries).


## 4. Python Simulation: Simplified MPC

In [ ]:

from cvxopt import matrix, solvers

# Parameters
N = 10   # horizon
Q = 1.0  # error weight
R = 0.1  # control effort weight
offset = 1.0
dt = 0.1

lane_offset_mpc = []
steering_mpc = []
time = np.arange(0, T, dt)

for t in time:
    # Quadratic cost: minimize offset^2 + steer^2
    P = matrix(np.diag([Q]*N + [R]*N))
    q = matrix([offset*Q]*N + [0]*N)

    # Constraints: -1 <= steer <= 1
    G = matrix(np.vstack([-np.eye(2*N), np.eye(2*N)]))
    h = matrix(np.hstack([np.ones(2*N), np.ones(2*N)]))

    sol = solvers.qp(P, q, G, h)
    u = sol['x'][N]   # take first steering command

    steer = float(u)
    offset -= steer * dt

    lane_offset_mpc.append(offset)
    steering_mpc.append(steer)

# Plot
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(time, lane_offset_mpc, label="MPC Lane Offset (m)")
plt.xlabel("Time (s)")
plt.ylabel("Offset (m)")
plt.title("MPC Lane Offset Correction")
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)
plt.plot(time, steering_mpc, label="MPC Steering Command")
plt.xlabel("Time (s)")
plt.ylabel("Steer (arb. units)")
plt.title("MPC Steering Output")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()



## 5. Observations

- **PID Controller**
  - Easy to implement, works well for small errors.
  - Struggles with sharp turns or constraints.
- **MPC Controller**
  - Considers future trajectory (prediction horizon).
  - Handles constraints explicitly.
  - Computationally heavier than PID.

### ✅ Conclusion
- PID is good for a baseline ADAS controller.
- MPC provides improved performance for advanced ADAS and autonomous systems.
